In [54]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import xgboost as xgb
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

from rdkit import Chem
from rdkit.Chem import Descriptors
from mordred import Calculator, descriptors

# Feature Selection: Обработка и отбор молекулярных дескрипторов
В этом блокноте происходит отбор ключевых дескрипторов для дальнейшего обучения модели. Молекулярные дескрипторы были отобраны из пакетов RDkit и Mordred. 

Для оптимизации признакового пространства был использован комбинированный подход, сочетающий два эффективных метода:
1) **Feature Importance на базе XGBoost.**
В качестве первого этапа применялся метод градиентного бустинга для обучения модели, позволяющей оценить важность каждого признака. Эта метрика, полученная через атрибут feature_importance, позволяет количественно определить вклад каждого признака в снижение ошибки модели, что даёт возможность выделить наиболее релевантные из них для дальнейшего использования.

2) **Библиотека Boruta.**
Для дополнения использован метод Boruta, который представляет собой обёртку для алгоритма Random Forest и основан на статистическом тестировании важности признаков. В процессе работы с Boruta каждый признак сравнивается с его случайно перемешанной копией (shadow feature), что позволяет выявить признаки, которые действительно оказывают значительное влияние на модель и исключить шумовые данные. Признаки, успешно проходящие тест на важность, получают метку "отобран", а все остальные - помечаются как нерелевантные. 

## Очистка данных
Стандартный pipeline очистки, включающий:
1) Удаление выбросов IQR-методом по целевому признаку LogP.
2) Фильтрация некорректных SMILES.
3) Удаление дубликатов SMILES.

In [55]:
def iqr_remove_outliers(df):
    logp_values = df['LogP']

    Q1 = np.percentile(logp_values, 25)
    Q3 = np.percentile(logp_values, 75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_clear = df[(logp_values >= lower_bound) & (logp_values <= upper_bound)]
    print(f"Number of outliers removed: {len(df) - len(df_clear)}")
    return df_clear


def remove_invalid_molecules(df):
    invalid_smiles_indices = []
    for index, row in df.iterrows():
        mol = Chem.MolFromSmiles(row['SMILES'])
        if mol is None:
            invalid_smiles_indices.append(index)

    df_clear = df.drop(invalid_smiles_indices).reset_index(drop=True)
    print(f"Number of invalid SMILES removed: {len(invalid_smiles_indices)}")
    return df_clear


def remove_duplicate_molecules(df):
    smiles_counts = df['SMILES'].value_counts()
    duplicates = smiles_counts[smiles_counts > 1].index
    df_clear = df[~df['SMILES'].isin(duplicates)]

    print(f"Number of duplicate SMILES removed: {len(duplicates)}")
    return df_clear

In [56]:
df = pd.read_csv('./data/final_train_data80.csv')
df = iqr_remove_outliers(df)
df = remove_invalid_molecules(df)
df = remove_duplicate_molecules(df)

Number of outliers removed: 0
Number of invalid SMILES removed: 0
Number of duplicate SMILES removed: 0


## Вычисление молекулярных дескрипторов
Для описания химической структуры молекул были использованы два набора дескрипторов:

- RDKit-дескрипторы
Содержат топологические, физико-химические и геометрические характеристики.

- Mordred-дескрипторы
Обширный набор (~1800 признаков), включая более редкие и специализированные характеристики.

In [57]:
def smiles_to_mols(smiles_list):
    return [Chem.MolFromSmiles(smi) for smi in smiles_list if Chem.MolFromSmiles(smi) is not None]

def compute_rdkit_descriptors(mol_list):
    return pd.DataFrame([{desc[0]: desc[1](mol) for desc in Descriptors.descList} for mol in mol_list])

def compute_mordred_descriptors(mol_list):
    calc = Calculator(descriptors, ignore_3D=True)
    results = []
    for mol in tqdm(mol_list, desc="Calculating Mordred descriptors"):
        desc = calc(mol)
        results.append(dict(desc))
    df = pd.DataFrame(results)
    return df

def clean_data(df):
   df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
   df = df.select_dtypes(include=[np.number])
   df_clear = df.rename(str, axis="columns") 
   return df_clear

In [58]:
smiles_list = df.loc[:, 'SMILES'].values
targets = df.loc[:, 'LogP'].values

mols_list = smiles_to_mols(smiles_list)
rdkit_df = compute_rdkit_descriptors(mols_list)
mordred_df = compute_mordred_descriptors(mols_list)

rdkit_x = clean_data(rdkit_df)
mordred_x = clean_data(mordred_df)
y = np.array(targets[:len(rdkit_x)])

Calculating Mordred descriptors: 100%|██████████| 10/10 [00:01<00:00,  9.60it/s]


## Отбор признаков
Дескрипторы, извлечённые с помощью RDKit и Mordred, обрабатываются по отдельности, поскольку некоторые параметры, вычисляемые этими библиотеками, являются идентичными. В случае их совместного использования возникает высокая корреляция между признаками, что приводит к мультиколлинеарности. Это негативно сказывается на отборе признаков. Разделение этих дескрипторов на два отдельных набора позволяет снизить избыточность признаков и обеспечить более точный и эффективный отбор, улучшая предсказательную способность моделей.

Результаты сохраняются в .csv файлах rdkit/mordred_selected_desc.csv. Финальный отбор происходит в ручном формате: отбираем самые влиятельные параметры, исключаем дубликаты и признаки с высокой корреляцией.

In [59]:
def xgboost_feature_ranking(X, y):
    X_scaled = StandardScaler().fit_transform(X)
    model = xgb.XGBRegressor(n_estimators=300, learning_rate=0.01, max_depth=5, random_state=666, n_jobs=-1)
    model.fit(X_scaled, y)
    importances = model.feature_importances_
    importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': importances
    }).sort_values(by='importance', ascending=False)
    return importance_df

In [60]:
def boruta_feature_ranking(X, y):
    X_scaled = StandardScaler().fit_transform(X)
    rf = RandomForestRegressor(n_jobs=-1, max_depth=5, random_state=666)
    boruta_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=666)
    boruta_selector.fit(X_scaled, y)
    selected = boruta_selector.support_
    ranked = boruta_selector.ranking_
    feature_ranks = pd.DataFrame({
        'feature': X.columns,
        'rank': ranked,
        'selected': selected
    }).sort_values(by='rank')
    return feature_ranks

In [61]:
rdkit_xgb_importances = xgboost_feature_ranking(rdkit_x, y)
rdkit_boruta_results = boruta_feature_ranking(rdkit_x, y)

mordred_xgb_importances = xgboost_feature_ranking(mordred_x, y)
mordred_boruta_results = boruta_feature_ranking(mordred_x, y)

In [62]:
merged_df = pd.merge(rdkit_xgb_importances, rdkit_boruta_results, on="feature", how="outer")
merged_df = merged_df.sort_values(by="importance", ascending=False)
merged_df.to_csv("./data/rdkit_selected_desc.csv", index=False)

merged_df = pd.merge(mordred_xgb_importances, mordred_boruta_results, on="feature", how="outer")
merged_df = merged_df.sort_values(by="importance", ascending=False)
merged_df.to_csv("./data/mordred_selected_desc.csv", index=False)